## Import Libraries

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta, Adagrad
from keras_tuner.tuners import RandomSearch

## Data Preprocessing

In [18]:
# Load datasets

data = np.load("data.npy")
target = np.load("target.npy")

In [19]:
# shapes of the datsets
print(data.shape)
print(target.shape)

(24966, 50, 50, 1)
(24966, 2)


## Split Train Test

In [20]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.1)

## Convolutional Neural Network Architecture

<figure>
    <center> <img src="./img/5.PNG"><center/>
</figure>

## Keras Tuner ( Hyper Parameter Optimization )

In [21]:
def build_model(parameters):
    
    model = Sequential()
    
    for i in range(parameters.Int("#layers ", 2,20)):      
        
        if(i==0):
            model.add(Conv2D(filters=parameters.Int("#conv layer " + str(i), min_value=32, max_value=512, step=32), 
                             kernel_size=(3, 3),
                             input_shape=data.shape[1:]))
            
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2, 2)))
         
        
        elif(i==1):
            model.add(Conv2D(filters=parameters.Int("#cov layer " + str(i), min_value=32, max_value=512, step=32), 
                             kernel_size=(3, 3)))
                      
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2, 2)))
       
                      
        elif(i==2):
            model.add(Flatten())
            model.add(Dropout(0.5))           
            
                      
        else:
            model.add(Dense(units=parameters.Int("#neuron layer " + str(i), min_value=32, max_value=512, step=32),
                           activation=parameters.Choice("activation function " + str(i), ["relu", "sigmoid", "tanh"])))
                      
            model.add(Dropout(parameters.Choice("drop probability ", ([0.2, 0.3, 0.4, 0.5]))))
                    
                      
    model.add(Dense(2, activation="softmax"))                 
    
    # compile
    model.compile(optimizer=parameters.Choice("optimizer ", ["adam", "adadelta", "adagrad"]), loss="binary_crossentropy")  
                    
    # model.compile(optimizer=Adam(parameters.Choice(learning_rate=[1e-2, 1e-3, 1e-4])), loss="mse")
    # model.compile(optimizer=AdaDelta(parameters.Choice(learning_rate=[1e-2, 1e-3, 1e-4])), loss="mse")
    # model.compile(optimizer=AdaGrade(parameters.Choice(learning_rate=[1e-2, 1e-3, 1e-4])), loss="mse")
                      
    return model      

In [22]:
tuner = RandomSearch(build_model, objective="val_loss", max_trials=3, executions_per_trial=2, directory="project-CNN-Cat-Dog", 
                     project_name="Cat-Dog")

In [23]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
#layers  (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
#conv layer 0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
#cov layer 1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
optimizer  (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adadelta', 'adagrad'], 'ordered': False}


## Search Best Parameters For The Model

In [ ]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
13                |13                |#layers 
224               |224               |#conv layer 0
128               |128               |#cov layer 1
adam              |adam              |optimizer 

Epoch 1/10
202/703 ━━━━━━━━━━━━━━━━━━━━ 1:03 127ms/step - loss: 0.6939

In [ ]:
tuner.results_summary() 

In [ ]:
best_model = tuner.get_best_models()
print(f"Best Model: {best_model[0].summary()} ")

## CNN Architecture

In [ ]:
model = Sequential()
                     
model.add(Conv2D(256, (3, 3), input_shape=data.shape[1:]))  
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# The first CNN layer followed by Relu and MaxPooling2D Layer

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# The second CNN layer followed by Relu and MaxPooling2D Layer

model.add(Flatten())
# Flatten layer to stack the output convolutions from second convolutional layer
model.add(Dropout(0.5))  
# For reduce overfitting
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
# The final layer with two outputs neurons for two categories
model.add(Dense(2, activation="softmax")) 
# softmax used for claasification problem

# compile 
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# summary
model.summary()

## Train The CNN

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

## Visualizing The Loss And The Accuracy of Trained CNN

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

## Evaluating The CNN Using Testing Data

In [ ]:
loss_accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss_accuracy[0]} \nAccuracy: {loss_accuracy[1]}")